In [ ]:
'''
    该文件用于改变原运行的结构，将原来的脚本分解进行
    用于直接进行输入输出，但是目前存在问题
'''


import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Subset

from sklearn.preprocessing import MinMaxScaler

from util.env import get_device, set_device
from util.preprocess import build_loc_net, construct_data
from util.net_struct import get_feature_map, get_fc_graph_struc
from util.iostream import printsep

from datasets.TimeDataset import TimeDataset


from models.GDN import GDN

from train import train
from test import test
from evaluate import get_err_scores, get_best_performance_data, get_val_performance_data, get_full_err_scores

import sys
from datetime import datetime

import os
import argparse
from pathlib import Path

import matplotlib.pyplot as plt

import json
import random

from imblearn.under_sampling import RandomUnderSampler

args = {}
args['batch'] = 32
args["epoch"] = 100
args["slide_win"] = 48
args["dim"] = 32
args["slide_stride"] = 1
args["comment"] = 'incident_network'
args["seed"] = 5
args["out_layer_num"] = 1
args["out_layer_inter_dim"] = 128
args["decay"] = 0
args["val_ratio"] = 0.2
args["topk"] = 6

# env_args
args["save_path"] = 'incident_network'
args["dataset"] = 'incident_network'
args["report"] = 'best'
args["device"] = 'cpu'
args["load_model_path"] = 'pretrained/incident_network/best_01|26-11:47:49.pt'


train_config = {
    'batch': args['batch'],
    'epoch': args["epoch"],
    'slide_win': args["slide_win"],
    'dim': args["dim"],
    'slide_stride': args["slide_stride"],
    'comment': args["comment"],
    'seed': args["seed"],
    'out_layer_num': args["out_layer_num"],
    'out_layer_inter_dim': args["out_layer_inter_dim"],
    'decay': args["decay"],
    'val_ratio': args["val_ratio"],
    'topk': args["topk"],
}

env_config = {
    'save_path': args["save_path"],
    'dataset': args["dataset"],
    'report': args["report"],
    'device': args["device"],
    'load_model_path': args["load_model_path"]
}

feature_map = get_feature_map(args["dataset"])
fc_struc = get_fc_graph_struc(args["dataset"])
columns = ["nds_ids",
           "travel_time",
           "state",
           "time_slice",
           "date",
           "month",
           "day",
           "hour",
           "minute",
           "speed"]

fc_edge_index = build_loc_net(fc_struc, columns, feature_map=feature_map)
fc_edge_index = torch.tensor(fc_edge_index, dtype=torch.long)
edge_index_sets = [fc_edge_index]

model = GDN(edge_index_sets, len(feature_map),
            dim=train_config['dim'],
            input_dim=train_config['slide_win'],
            out_layer_num=train_config['out_layer_num'],
            out_layer_inter_dim=train_config['out_layer_inter_dim'],
            topk=train_config['topk']
            ).to(args["device"])

model.load_state_dict(torch.load(args["load_model_path"]))
model = model.to(args["device"])

input = data_all_1874_21_17_val.to_numpy()
# input = input[np.newaxis, :, :]
input_x = input[:, :-1].T
input_label = input[:, -1].T

input_x = torch.tensor(input_x).double()
input_label = torch.tensor(input_label).double()

X, y, label = process(input_x, input_label)
input = torch.tensor(input)
X = X.to(torch.float32)
y = y.to(torch.float32)
label = label.to(torch.float32)

predicted = model(X,
                  fc_edge_index).float().to(args["device"])
predicted

t_test_predicted_list = predicted.tolist()
t_test_ground_list = y.tolist()
t_test_labels_list = label.tolist()

np_test_result = np.array(
    [t_test_predicted_list, t_test_ground_list, t_test_labels_list])
# test_labels = np_test_result[2, :, 0].tolist()
test_labels = t_test_labels_list

feature_num = len(np_test_result[-1])
all_scores = None

# 计算到这里无法在继续下去了，没有搞明白这里的分数的计算方法
for i in range(feature_num):
    test_re_list = np_test_result[:2, :, i]
    scores = get_err_scores(test_re_list, test_re_list)
    all_scores = np.vstack((
        all_scores,
        scores
    ))

top1_best_info = get_best_performance_data(all_scores, test_labels, topk=1)
